# 🗄️ Task 3 – Oracle Database Ingestion  
📘 Version: 2025-06-11

Persist cleaned and enriched Google Play reviews into an Oracle database for three Ethiopian banks (CBE, BOA, Dashen), establishing a normalized schema and loading data for downstream analytics.

### This notebook/module covers:
- Defining and deploying the relational schema (`banks`, `reviews` with foreign key) in Oracle  
- Connecting to Oracle via `src/db/oracle_connector.py` with robust, environment-driven credentials  
- Dropping and re-creating tables defensively, ignoring “table does not exist” errors  
- Extracting distinct banks to populate the `banks` table and mapping names to surrogate keys  
- Bulk-loading `data/outputs/reviews_enriched_all.csv` into the `reviews` table using batched `executemany`  
- Committing transactions and handling tablespace quota programmatically  
- Cleaning up connections and cursors with full error handling and inline debug logging  


In [1]:
# ------------------------------------------------------------------------------
# 🛠 Ensure Notebook Runs from Project Root (for src/ imports to work)
# ------------------------------------------------------------------------------

import os
import sys

# If running from /notebooks/, move up to project root
if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")
    print("📂 Changed working directory to project root")

# Add project root to sys.path so `src/` modules can be imported
project_root = os.getcwd()
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"✅ Added to sys.path: {project_root}")

# Optional: verify file presence to confirm we're in the right place
expected_path = "data/raw"
print(
    "📁 Output path ready"
    if os.path.exists(expected_path)
    else f"⚠️ Output path not found: {expected_path}"
)

📂 Changed working directory to project root
✅ Added to sys.path: c:\Users\admin\Documents\GIT Repositories\b5w2-customer-ux-analytics-challenge
📁 Output path ready


## 🚀 Deploy Schema & Load Enriched Reviews

This step orchestrates the end‐to‐end ingestion of enriched Google Play reviews into Oracle, ensuring an idempotent, robust setup:

- Instantiates `OracleConnector` (loads `.env`, validates credentials, and configures logging).  
- Idempotently drops existing `reviews` and `banks` tables, skipping if they don’t exist.  
- Idempotently creates the `banks` (dimension) and `reviews` (fact) tables with proper foreign key.  
- Bulk‐loads `reviews_enriched_all.csv` from `data/outputs/`, parsing dates and numeric fields, and mapping bank names to surrogate keys.  
- Commits all transactions and closes the connection with graceful error handling.  

This single cell drives the Task 3 database pipeline, leveraging a fully OOP connector for clarity, maintainability, and repeatability.  


In [2]:
# ------------------------------------------------------------------------------
# 🚀 Task 3 – Deploy Schema & Load Enriched Reviews
# ------------------------------------------------------------------------------

from src.db.oracle_connector import OracleConnector

try:
    connector = OracleConnector(verbose=True)  # initialize connector
    connector.drop_tables()  # drop existing tables
    connector.create_schema()  # create banks & reviews schema
    connector.load_data(
        csv_relative="data/outputs/reviews_enriched_all.csv"
    )  # bulk-load data
    connector.close()  # close connection
    print("🎉 Task 3 complete: schema deployed and data loaded.")
except Exception as e:
    print(f"❌ Task 3 failed: {e}")
    # connector may be in inconsistent state; ensure cleanup
    try:
        connector.close()
    except:
        pass

2025-06-11 07:19:23,429 DEBUG Env loaded → user=nablop, dsn=localhost:1521/xepdb1
2025-06-11 07:19:23,552 INFO ✅ Connected as nablop
2025-06-11 07:19:23,558 DEBUG Table 'reviews' exists? True
2025-06-11 07:19:23,705 INFO ➖ Dropped existing table 'reviews'
2025-06-11 07:19:23,710 DEBUG Table 'banks' exists? True
2025-06-11 07:19:23,780 INFO ➖ Dropped existing table 'banks'
2025-06-11 07:19:23,784 DEBUG Table 'banks' exists? False
2025-06-11 07:19:23,880 INFO ✅ Created table 'banks'
2025-06-11 07:19:23,883 DEBUG Table 'reviews' exists? False
2025-06-11 07:19:23,959 INFO ✅ Created table 'reviews'
2025-06-11 07:19:24,019 DEBUG Parsed 1200 rows, 3 banks
2025-06-11 07:19:24,822 INFO Inserted 3 banks
2025-06-11 07:19:26,374 INFO Loaded 1200 reviews
2025-06-11 07:19:26,385 INFO 🔒 Connection closed


🎉 Task 3 complete: schema deployed and data loaded.
